# <font color=red>HW02-q1 Vision Course, Panorama</font>
This is the notebook for **q1.py** that is the implementation of **Video Panorama and Processing**. <br>
The code is written by **Asal Mehradfar** with student number **96105434**.

## <font color=orange>Description</font>

*   I used the images with their best resolution in the first step, but
for the other parts running the codes took a long time on my 8 RAM LAPTOP!
So I decreased the resolution to quarter of the original one, haft each dimension.
<br>
*   At the 7th step, my results are not PERFECT. It can not detect the black cars
which are far from the camera because of the similarity between their color
and the background color, I mean the roads.
* the zip file of the results are on my google drive in the link below:<br>
[HW2_Results_96105434](https://drive.google.com/file/d/1E5dSZ2XjAk92DSmzueIJttUfc9p9QAc1/view?usp=sharing)
*   I partitioned my code into 7 functions for the 7 parts like p1, p2,$$\dots$$
and also other useful functions. you can run each of them by uncommenting the main part.
*   Pay attention that capture_frames() function makes some folders and frames
that we need for the code, so make sure that you run them in the order of the main part.
* The report of what I have done is written in each of the p1, p2, $$\dots$$ functions.

### <font color=yellow>Imports</font>

In [32]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tqdm
from scipy import stats
from scipy import ndimage as nd

### <font color=yellow>Parameters</font>

*   NUM_FRAMES = 900
*   NUM_REF_FRAME = 450

In [2]:
NUM_FRAMES = 900
NUM_REF_FRAME = 450
R = 0.7
MAX_ITERATIONS = 500

### <font color=yellow>Functions</font>

In [7]:
def get_img(path):
    """
    Read the image file from the path and change it from BGR to RGB
    pay attention that in open-cv colorful images are BGR **NOT** RGB

    Inputs:
    --> path: path for the image
    Outputs:
    ==> img: the RGB image
    """
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def plot_img(img, path=None):
    """
    Plot a colorful image and save it if needed

    Inputs:
    --> img: the desired image
    --> path: the default value is None, if it is given the image will be saved in the path
    Outputs:
    ==> Nothing, the image will be plotted
    """
    fig = plt.figure(figsize=(16, 8))
    plt.imshow(img.astype(np.uint8))
    plt.axis('off')
    if path is not None:
        fig.savefig(path)
    else:
        plt.show()

def scaling_img(img):
    """
    return the scaled image usually for saving

    Inputs:
    --> img: the desired image for scaling
    Outputs:
    ==> scaled_img: we assume that the minimum of the image is zero
        so we scale it by division by its maximum and multiplying it by 255.0
    """
    scaled_img = 255.0 * img / np.max(img)
    return scaled_img

def save_img(array, path):
    """
    save the input image in the desired path

    Inputs:
    --> array: the array of an image
    --> path: the desired path for saving the image
    Outputs:
    ==> Nothing, just saving the image
    """
    scaled = scaling_img(array).astype(np.uint8)
    img = Image.fromarray(scaled)
    img.save(path)

def capture_frames(path, save_path):
    """
    Save NUM_FRAMES video frames in a folder

    Inputs:
    --> path: the filename of the video
    --> save_path: the folder name of the saving frames
    Outputs:
    ==> Nothing, frames will be saved with 3 digit names
    """
    os.mkdir(save_path)
    vid = cv2.VideoCapture(path)

    for i in range(NUM_FRAMES):
        ret, frame = vid.read()
        if not ret:
            break
        cv2.imwrite(save_path + '/' + str(i+1).zfill(3) + '.jpg', frame)

    vid.release()
    cv2.destroyAllWindows()

def capture_frames_low_res():
    """
    Save NUM_FRAMES video frames in a folder with low resolution
    we down sample each dim to half of its previous form
    so we decrease resolution to quarter of the original

    Inputs:
    --> Nothing
    Outputs:
    ==> Nothing, frames will be saved with 3 digit names
    """
    os.mkdir('VideoFramesLowRes')
    vid = cv2.VideoCapture('video.mp4')

    for i in range(NUM_FRAMES):
        ret, frame = vid.read()
        if not ret:
            break
        size = (int(frame.shape[1] / 2), int(frame.shape[0] / 2))
        new_frame = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
        cv2.imwrite('VideoFramesLowRes/' + str(i+1).zfill(3) + '.jpg', new_frame)

    vid.release()
    cv2.destroyAllWindows()

def find_homography(img1, img2):
    """
    Finding match points between two images by RANSAC Algorithm

    Inputs:
    --> img1: the first desired image
    --> img2: the second desired image
    Outputs:
    ==> H: the 3*3 homography matrix
    """

    sift = cv2.SIFT_create()

    # find all key points
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # find all matches
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    # find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < R * n.distance:
            good_matches.append([m])

    kp1_match = [kp1[m[0].queryIdx] for m in good_matches]
    kp2_match = [kp2[m[0].trainIdx] for m in good_matches]

    # apply RANSAC
    src_pts = np.float32([kp.pt for kp in kp1_match]).reshape(-1, 1, 2)
    des_pts = np.float32([kp.pt for kp in kp2_match]).reshape(-1, 1, 2)
    H,_ = cv2.findHomography(src_pts, des_pts, cv2.RANSAC, maxIters=MAX_ITERATIONS)

    return np.linalg.inv(H)

def perspective(img, H):
    """
    computing the homography of the image with matrix H

    Inputs:
    --> img: the desired image
    --> H: the homography matrix
    Outputs:
    ==> img_homography: the output image
    """
    x_min, y_min, x_max, y_max = corners_homography(img, H)
    [x_offset, y_offset] = [-x_min, -y_min]
    [x_size, y_size] = [x_max-x_min, y_max-y_min]
    offset = np.array([[1, 0, x_offset], [0, 1, y_offset], [0, 0, 1]])
    img_homography = cv2.warpPerspective(img, np.dot(offset, H), (x_size, y_size))
    return img_homography

def panorama_two_images(img1, img2, H):
    """
    computing the panorama of two input images with matrix H on the second image
    the first image will be up to the second one in the union

    Inputs:
    --> img1: the first desired image
    --> img2: the second desired image
    --> H: the homography matrix
    Outputs:
    ==> img: the output panorama image
    """
    x_min, y_min, x_max, y_max = corners_homography(img2, H)
    [x_offset, y_offset] = [-x_min, -y_min]
    [x_size, y_size] = [max(img1.shape[1] + max(0, x_offset), x_max-x_min),
                        max(img1.shape[0] + max(0, y_offset), y_max-y_min)]
    offset = np.array([[1, 0, x_offset], [0, 1, y_offset], [0, 0, 1]])
    img = cv2.warpPerspective(img2, np.dot(offset, H), (x_size, y_size))
    img[y_offset: img1.shape[0] + y_offset, x_offset: img1.shape[1] + x_offset, :] = img1
    return img

def corners_homography(img, H):
    """
    computing the min and max of the corners of an image
    after applying the homography matrix without offset
    we use these points to find offset and size of the homography

    Inputs:
    --> img: the desired image
    --> H: the homography matrix
    Outputs:
    ==> x_min: minimum x after homography
    ==> y_min: minimum y after homography
    ==> x_max: maximum x after homography
    ==> y_max: maximum y after homography
    """
    height, width, _ = img.shape
    corners = [np.array([[0, 0, 1]]).transpose(),
            np.array([[width-1, 0, 1]]).transpose(),
            np.array([[0, height-1, 1]]).transpose(),
            np.array([[width-1, height-1, 1]]).transpose()]
    [x_min, y_min, x_max, y_max] = [-1 for _ in range(4)]
    for c in corners:
        m = np.matmul(H, c)
        if x_min == -1:
            [x_min, y_min, x_max, y_max] = [int(m[0] / m[2]),
                                            int(m[1] / m[2]),
                                            int(m[0] / m[2]),
                                            int(m[1] / m[2])]
        else:
            [x_min, y_min, x_max, y_max] = [min(x_min, int(m[0] / m[2])),
                                            min(y_min, int(m[1] / m[2])),
                                            max(x_max, int(m[0] / m[2])),
                                            max(y_max, int(m[1] / m[2]))]
    return x_min, y_min, x_max, y_max

def compute_H_to_450():
    """
    computing the homography matrix for all the images to the reference image:
    moving images less than 90 to 90 and then to 270 and at last 450,
    moving images less than 270 and more than 90 to 270 and then 450,
    moving images less than 630 and more than 270 directly to 450,
    moving images less than 810 and more than 630 to 630 and then 450,
    moving image more than 810 to 810 and then to 630 and at last 450.

    Inputs:
    --> Nothing
    Outputs:
    ==> H: a 900*3*3 matrix, each row is a homography matrix
        for one of the frames to the reference image
    """
    H = np.zeros((NUM_FRAMES, 3, 3))
    ref_images, H[89, :, :], H[269, :, :], H[449, :, :], H[629, :, :], H[809, :, :] = compute_key_frames_H()

    for i in tqdm.tqdm(range(NUM_FRAMES)):
        if i == 89 or i == 269 or i == 449 or i == 629 or i == 809:
            continue
        elif i < 89:
            H[i, :, :] = np.matmul(H[89, :, :], find_homography(ref_images[0],
                            get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg')))
        elif i<269:
            H[i, :, :] = np.matmul(H[269, :, :], find_homography(ref_images[1],
                            get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg')))
        elif i<629:
            H[i, :, :] = find_homography(ref_images[2],
                            get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg'))
        elif i<809:
            H[i, :, :] = np.matmul(H[629, :, :], find_homography(ref_images[3],
                            get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg')))
        else:
            H[i, :, :] = np.matmul(H[809, :, :], find_homography(ref_images[4],
                            get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg')))

    return H

def compute_key_frames_H():
    """
    computing key frames and homography of them

    Inputs:
    --> Nothing
    Outputs:
    ==> key_images: the list of 5 key frames
    ==> H90: the homography of 90 to 450
    ==> H180: the homography of 180 to 450
    ==> H450: the homography of 450 to 450
    ==> H630: the homography of 630 to 450
    ==> H810: the homography of 810 to 450
    """
    direc = 'VideoFramesLowRes/'
    key_images = [
        get_img(direc + '090.jpg'),
        get_img(direc + '270.jpg'),
        get_img(direc + '450.jpg'),
        get_img(direc + '630.jpg'),
        get_img(direc + '810.jpg')
    ]
    H90 = np.matmul(find_homography(key_images[2], key_images[1]), find_homography(key_images[1], key_images[0]))
    H270 = find_homography(key_images[2], key_images[1])
    H450 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    H630 = find_homography(key_images[2], key_images[3])
    H810 = np.matmul(find_homography(key_images[2], key_images[3]), find_homography(key_images[3], key_images[4]))

    return key_images, H90, H270, H450, H630, H810

def video_corners_homography():
    """
    computing the minimum and maximum coordinates for all the frames after homography to
    the reference image 450

    Inputs:
    --> Nothing
    Outputs:
    ==> H: a 900*3*3 matrix, each row is a homography matrix
        for one of the frames to the reference image
    ==> x_min: minimum x after homography for all of the frames
    ==> y_min: minimum y after homography for all of the frames
    ==> x_max: maximum x after homography for all of the frames
    ==> y_max: maximum y after homography for all of the frames
    """
    H = compute_H_to_450()
    [x_min, y_min, x_max, y_max] = corners_homography(get_img('VideoFramesLowRes/' + str(1).zfill(3) +'.jpg'),
                                                        H[0, :, :])
    for i in range(1, NUM_FRAMES):
        a = corners_homography(get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg'),
                                                        H[i, :, :])
        x_min = min(x_min, a[0])
        y_min = min(y_min, a[1])
        x_max = max(x_max, a[2])
        y_max = max(y_max, a[3])

    return H, x_min, y_min, x_max, y_max

def key_frames_corners_homography():
    """
    computing corers and homography for 5 key frames
    computing the minimum and maximum coordinates for all of the key frames
    after homography to the reference image 450

    Inputs:
    --> Nothing
    Outputs:
    ==> key_frames: the list of 5 key frames
    ==> H: the 5*3*3 array of homography
    ==> x_min: minimum of x
    ==> y_min: minimum of y
    ==> x_max: maximum of x
    ==> y_max: maximum of y
    """
    H = np.zeros((5, 3, 3))
    key_frames, H[0, :, :], H[1, :, :], H[2, :, :], H[3, :, :], H[4, :, :] = compute_key_frames_H()
    [x_min, y_min, x_max, y_max] = corners_homography(key_frames[0], H[0, :, :])

    for i in range(1, 5):
        a = corners_homography(key_frames[i],H[i, :, :])
        x_min = min(x_min, a[0])
        y_min = min(y_min, a[1])
        x_max = max(x_max, a[2])
        y_max = max(y_max, a[3])

    return key_frames, H, x_min, y_min, x_max, y_max

def panorama_key_frames(img1, img2):
    """
    computing the panorama of two images
    we use a squared difference matrix of two images for computing a value as cost
    first we find the union of two images, we crop the pixels of the union part and
    make a cost matrix of that size
    for this purpose I initialized the first line of the union then used dp algorithm
    for computing the cost matrix
    by this approach, I found the path with the lowest cost between two images
    and used the pixels of the first image for the right side of the panorama and
    the pixels of the second image for the left side

    Inputs:
    --> img1: the first desired image (base one)
    --> img2: the second desired image
    Outputs:
    ==> out_img: the panorama of two images
    """

    [fr1, fr2] = [img1.copy(), img2.copy()]
    mask = np.zeros(img1.shape)
    diff = np.sum(((img1 - img2)/100) * ((img1 - img2)/100), axis=2)

    # making the union matrix
    img1[np.any(img1 != [0, 0, 0], axis=-1)] = [1, 1, 1]
    img2[np.any(img2 != [0, 0, 0], axis=-1)] = [1, 1, 1]

    union = img1 * img2

    # computing the cost matrix for the union part
    (x, y, _) = np.nonzero(union)
    [x_min, y_min, x_max, y_max] = [min(x), min(y), max(x), max(y)]
    cropped_img = np.ones((x_max-x_min, y_max-y_min)) * np.inf

    # initialization
    for i in range(cropped_img.shape[1]):
        for j in range(cropped_img.shape[0]):
            if union[x_min + j, y_min + i, 0] == 1:
                if j < cropped_img.shape[0] / 4:
                    cropped_img[j, i] = diff[x_min + j, y_min + i]
                break
    # computing cost
    for i in range(1, cropped_img.shape[0]):
        for j in range(1, cropped_img.shape[1]-1):
            m = min(cropped_img[i-1, j-1], cropped_img[i-1, j], cropped_img[i-1, j+1])
            if m != np.inf:
                cropped_img[i, j] = diff[x_min + i, y_min + j] + m
    # omitting the last rows of the matrix
    for i in range(cropped_img.shape[1]-1, 0, -1):
        for j in range(cropped_img.shape[0]-1, 0, -1):
            if union[x_min + j, y_min + i, 0] == 1:
                if j > 3 * cropped_img.shape[0] / 4:
                    cropped_img[j, i] = np.inf
                break
            else:
                cropped_img[j, i] = np.inf

    # finding the path between two parts of panorama
    val = np.inf
    for i in range(cropped_img.shape[1]-1, 0, -1):
        for j in range(cropped_img.shape[0]-1, 0, -1):
            if cropped_img[j, i] != np.inf:
                if j > 3 * cropped_img.shape[0] / 4:
                    if cropped_img[j, i] < val:
                        [x, y, val] = [j+x_min, i+y_min, cropped_img[j, i]]
                break

    # computing the mask
    for i in range(img1.shape[0]-1, x-1, -1):
        for j in range(y):
            mask[i, j, :] = [1, 1, 1]

    while x > 0 and y > 0 and cropped_img[x-x_min, y-y_min] != np.inf:
        m = min(cropped_img[x-1-x_min, y-1-y_min],
                cropped_img[x-1-x_min, y-y_min],
                cropped_img[x-1-x_min, y+1-y_min])
        if m == cropped_img[x-1-x_min, y-1-y_min]:
            [x, y] = [x-1, y-1]
        elif m == cropped_img[x-1-x_min, y-y_min]:
            [x, y] = [x-1, y]
        else:
            [x, y] = [x-1, y+1]
        for j in range(y):
            mask[x, j, :] = [1, 1, 1]

    for i in range(x):
        for j in range(y):
            mask[i, j, :] = [1, 1, 1]

    out_img = mask * fr1 + (1-mask) * fr2

    return out_img


### <font color=yellow>Main Part Functions</font>

In [96]:
def p1(flag=False):
    """
    part one
    drawing a red rectangle and then applying the H inverse,
    after that making a bigger matrix and first put the 900 frame and then 450
    on that.

    Inputs:
    --> flag: if true saving results, else plotting
    Outputs:
    ==> Nothing, just saving or plotting results
    """
    img1 = get_img('VideoFrames/450.jpg')
    img2 = get_img('VideoFrames/270.jpg')
    H = find_homography(img1, img2)
    H_inv = np.linalg.inv(H)

    points = [(500, 500), (1000, 500), (1000, 1000), (500, 1000)]
    out_points = []
    color = (255,0,0)
    thickness = 5

    img1_new = img1.copy()
    img2_new = img2.copy()

    img1_new = cv2.rectangle(img1_new, points[0], points[2], color, thickness)

    for i in range(len(points)):
        p = np.array([[points[i][0], points[i][1], 1]]).transpose()
        m = np.matmul(H_inv, p)
        out_points.append((int(m[0] / m[2]), int(m[1] / m[2])))

    for i in range(len(out_points)):
        img2_new = cv2.line(img2_new, out_points[i], out_points[(i+1)%4], color, thickness)

    img = panorama_two_images(img1, img2, H)

    if flag:
        save_img(img1_new, 'res01-450-rect.jpg')
        save_img(img2_new, 'res02-270-rect.jpg')
        save_img(img, 'res03-270-450-panorama.jpg')
    else:
        plot_img(img1_new)
        plot_img(img2_new)
        plot_img(img)

def p2(flag=False):
    """
    part two
    I explained my approach in the functions used below,
    just as a short review, I moved all the 5 key frames to the reference frame
    which was 450, and then used a dp algorithm to merge them.

    Inputs:
    --> flag: if true saving results, else plotting
    Outputs:
    ==> Nothing, just saving or plotting results
    """
    key_frames, H, x_min, y_min, x_max, y_max = key_frames_corners_homography()
    [x_offset, y_offset] = [-x_min, -y_min]
    (x_size, y_size) = (x_max-x_min, y_max-y_min)
    offset = np.array([[1, 0, x_offset], [0, 1, y_offset], [0, 0, 1]])
    homography_frames = [cv2.warpPerspective(key_frames[i], np.dot(offset, H[i, :, :]), (x_size, y_size))
                         for i in range(5)]

    full_image = homography_frames[0]
    for i in range(1, 5):
        full_image = panorama_key_frames(full_image.copy(), homography_frames[i].copy())

    if flag:
        save_img(full_image, 'res04-key-frames-panorama.jpg')
    else:
        plot_img(full_image)

def p3():
    """
    part three
    here I easily apply the H matrix to each frame using a same offset
    computed before and write the frames in batches on the video.
    pay attention that open-cv consider images in BGR not RGB, so before writing
    we should consider this fact.

    Inputs:
    --> Nothing
    Outputs:
    ==> Nothing, just saving the panorama video
    """
    H, x_min, y_min, x_max, y_max = video_corners_homography()
    [x_offset, y_offset] = [-x_min, -y_min]
    size = (x_max-x_min, y_max-y_min)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('res05-reference-plane.mp4', fourcc, 30, size)
    frames = []

    for i in tqdm.tqdm(range(NUM_FRAMES)):
        img = get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg')
        offset = np.array([[1, 0, x_offset], [0, 1, y_offset], [0, 0, 1]])
        frame = cv2.warpPerspective(img, np.dot(offset, H[i,:,:]), size)
        frames.append(frame)
        if (i+1) % 30 == 0:
            for j in range(30):
                video.write(cv2.cvtColor(frames[j], cv2.COLOR_RGB2BGR))
            frames = []

    video.release()

def p4(flag= False):
    """
    part four
    I used the mode for computing the value of background pixels
    but the RAM of the laptop does not let us to directly compute the mode for all of
    the pixels of an image,
    so I made some batches and computed mode for them.
    for RAM 8, 100 is a good value for running and for RAM 12 250 works correctly.
    with this approach and computing mode for RGB separately, we see a lot of black pixels
    in the corners than happens because of camera moving.
    to solve this I take a sum of RGB and consider all the values less than a special threshold
    like 40 as black pixels and remove them from mode computing. now we have a better background
    image in the corners.

    but how do I remove black pixels from the mode:
    we know that all the images have the values from 0 to 255, so for each black point
    in an image I assigned a value more than 255 and different from others to black pixels
    and now because of the difference we are sure that the mode is not any of these pixels.

    Inputs:
    --> flag: if true saving results, else plotting
    Outputs:
    ==> Nothing, just saving or plotting results
    """

    batch = 100
    thresh = 40
    frame = get_img('HomographyFrames/' + str(1).zfill(3) +'.jpg')
    frames = np.zeros((NUM_FRAMES, frame.shape[0], batch, 3))

    background_img = np.zeros(frame.shape)

    for j in tqdm.tqdm(range(0, frame.shape[1]-batch, batch)):
        temp = np.zeros((frame.shape[0], batch, 3)) + 256
        for i in range(NUM_FRAMES):
            frame = get_img('HomographyFrames/' + str(i+1).zfill(3) +'.jpg')
            a = frame[:, j:j+batch, :]
            s = np.sum(a, axis=2)
            s = s < thresh
            (x, y) = np.nonzero(s)
            a[x, y, :] = temp[x, y, :]
            temp[x, y, :] = temp[x, y, :] + 1
            frames[i, :, :, :] = a
        background_img[:, j:j+batch, :] = stats.mode(frames, axis=0).mode

    # the last batch
    frames = np.zeros((NUM_FRAMES, frame.shape[0], frame.shape[1] % batch , 3))
    j = int(frame.shape[1] / batch) * batch
    temp = np.zeros((frame.shape[0], frame.shape[1] % batch, 3)) + 256
    for i in range(NUM_FRAMES):
        frame = get_img('HomographyFrames/' + str(i+1).zfill(3) +'.jpg')
        a = frame[:, j:, :]
        s = np.sum(a, axis=2)
        s = s < thresh
        (x, y) = np.nonzero(s)
        a[x, y, :] = temp[x, y, :]
        temp[x, y, :] = temp[x, y, :] + 1
        frames[i, :, :, :] = a
    background_img[:, j:, :] = stats.mode(frames, axis=0).mode

    if flag:
        save_img(background_img, 'res06-background-panorama.jpg')
    else:
        plot_img(background_img)

def p5():
    """
    part five

    I applied the inverse of the offset matrix computed before multiplied to H
    with the warp perspective on the background image for different frames
    I used the frame size of the extracted video for the size of the current video

    Inputs:
    --> Nothing
    Outputs:
    ==> Nothing, just saving the background video
    """
    H, x_min, y_min, x_max, y_max = video_corners_homography()
    [x_offset, y_offset] = [-x_min, -y_min]
    img = get_img('VideoFramesLowRes/' + str(1).zfill(3) +'.jpg')
    vid_size = (img.shape[1], img.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('res07-background-video.mp4', fourcc, 30, vid_size)
    frames = []

    for i in tqdm.tqdm(range(NUM_FRAMES)):
        img = get_img('res06-background-panorama.jpg')
        offset = np.array([[1, 0, x_offset], [0, 1, y_offset], [0, 0, 1]])
        frame = cv2.warpPerspective(img, np.linalg.inv(np.dot(offset, H[i,:,:])), vid_size)
        frames.append(frame)
        if (i+1) % 30 == 0:
            for j in range(30):
                video.write(cv2.cvtColor(frames[j], cv2.COLOR_RGB2BGR))
            frames = []

    video.release()

def p6():
    """
    part six
    first I load the original image and the background,
    then I make a difference matrix same as the previous parts using norm 2,
    here some individual points have big differences so I used a uniform filter
    to solve this problem,
    after that I normalize the difference matrix and use a threshold for emphasizing
    the points which are the foregrounds.
    now we should make the explained points red by adding 100 to their R and clipping
    them to 255.
    at last write the frames in batches on the video in a loop.

    Inputs:
    --> Nothing
    Outputs:
    ==> Nothing, just saving the video
    """
    img = get_img('VideoFramesLowRes/' + str(1).zfill(3) +'.jpg')
    vid_size = (img.shape[1], img.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('res08-foreground-video.mp4', fourcc, 30, vid_size)
    frames = []

    for i in tqdm.tqdm(range(NUM_FRAMES)):
        img1 = get_img('VideoFramesLowRes/' + str(i+1).zfill(3) +'.jpg').astype(np.float16)
        img2 = get_img('BackgroundFrames/' + str(i+1).zfill(3) +'.jpg').astype(np.float16)
        diff = np.sum(((img1 - img2)/100) * ((img1 - img2)/100), axis=2)
        diff = nd.uniform_filter(diff.astype(np.uint8), size=15, mode='constant')
        diff = (diff - np.min(diff)) / (np.max(diff) - np.min(diff))
        diff = diff > 0.01
        frame = img1.copy()
        frame[:, :, 0] = np.clip(frame[:, :, 0] + 100*diff, 0, 255)
        frames.append(frame.astype(np.uint8))
        if (i+1) % 30 == 0:
            for j in range(30):
                video.write(cv2.cvtColor(frames[j], cv2.COLOR_RGB2BGR))
            frames = []

    video.release()

def p7():
    """
    part seven
    I increased the video size by multiplying the x size of the image to 1.5
    in the warp perspective.
    pay attention that when we make the video wider, in the last frames of the 30sec video
    we will see a part of the video black, because we do not have any data in the frames
    for that part
    so I removed the frames after 630 that did not give us any useful data
    our final video now is around 21secs.

    Inputs:
    --> Nothing
    Outputs:
    ==> Nothing, just saving the wided background video
    """
    H, x_min, y_min, x_max, y_max = video_corners_homography()
    [x_offset, y_offset] = [-x_min, -y_min]
    img = get_img('VideoFramesLowRes/' + str(1).zfill(3) +'.jpg')
    vid_size = (int(img.shape[1]*1.5), img.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('res09-background-video-wider.mp4', fourcc, 30, vid_size)
    frames = []

    for i in tqdm.tqdm(range(630)):
        img = get_img('res06-background-panorama.jpg')
        offset = np.array([[1, 0, x_offset], [0, 1, y_offset], [0, 0, 1]])
        frame = cv2.warpPerspective(img, np.linalg.inv(np.dot(offset, H[i,:,:])), vid_size)
        frames.append(frame)
        if (i+1) % 30 == 0:
            for j in range(30):
                video.write(cv2.cvtColor(frames[j], cv2.COLOR_RGB2BGR))
            frames = []

    video.release()

### <font color=yellow>Main Part</font>

In [5]:
capture_frames('video.mp4', 'VideoFrames')
capture_frames_low_res()
p1(True)
# p2(True)
# p3()
# capture_frames('res05-reference-plane.mp4', 'HomographyFrames')
# p4(True)
# p5()
# capture_frames('res07-background-video.mp4', 'BackgroundFrames')
# p6()
# p7()